# Introduction

Python의 Model I/O 에 대해 학습함

# FewShotPromptTempalte

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# Shot :

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
)

prompt

chain = prompt | chat

chain.invoke({"country": "Spain"})

C:\Users\yunyg\AppData\Local\Temp\ipykernel_6988\2370880079.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


I know this:
- Capital: Madrid
- Language: Spanish
- Food: Paella and Tapas
- Currency: Euro

AIMessage(content='I know this:\n- Capital: Madrid\n- Language: Spanish\n- Food: Paella and Tapas\n- Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--cddb848f-2f49-4fed-aa1c-c311443b189b-0')

## FewShotChatMessagePromptTemplate

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]



In [4]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answer."),
        example_prompt,
        ("human", "What do you know about {country}?")
    ]
)

In [5]:
chain = final_prompt | chat
chain.invoke(
    {
        "country" : "Korea"
    }
)


        Korea refers to two countries:
        - South Korea (Seoul is the capital, language: Korean, currency: South Korean won)
        - North Korea (Pyongyang is the capital, language: Korean, currency: North Korean won)

AIMessage(content='\n        Korea refers to two countries:\n        - South Korea (Seoul is the capital, language: Korean, currency: South Korean won)\n        - North Korea (Pyongyang is the capital, language: Korean, currency: North Korean won)', additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run--c9438edd-95f9-48f5-a4f3-53a6353dc76b-0')

# RandomBasedExampleSelector

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model = "gpt-4.1-nano",
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]



In [7]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(examples)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]

In [8]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples
)

In [9]:
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"]
)

prompt.format(country = "Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'